<img style="float: right; margin: 0px 0px 15px 15px;" src="https://educationusa.state.gov/sites/default/files/wysiwyg/iteso_logo.jpg" width="520px" height="230px" />

# <span style="color: darkblue; ">Advanced Trading Strategies: _Statistical Arbitrage_</span>
`MICROSTRUCTURE AND TRADING SYSTEMS`

Juan Ramón Rocha López 

- Exp: 739950

Mariana Valenzuela Lafarga
    
- Exp: 749770

Repository on GitHub: [Link del repositorio](https://github.com/RaemonRoch/algo-trading)

### Strategy Description and Rationale

##### 1. Overview
- Strategy: Pairs trading (statistical arbitrage) that captures mean-reversion of the spread between two assets (e.g., GOOGL vs AMD). The execution flow and main backtest parameters (splits, fees, borrow rates, windows, thresholds) are orchestrated in main.py.

##### 2. Pairs trading approach
- Spread definition: model P1 as a linear combination of P2: P1 ≈ beta0 + beta1 * P2; the residual (spread = P1 - (beta0 + beta1*P2)) is traded. Trading logic (spread, z-score, thresholds, history) is implemented in MarketNeutralStrategy.
- Signals: compute z-score of spread over a rolling window. Enter when |z-score| > entry_threshold; exit when |z-score| < exit_threshold.
- Sizing and execution: StrategyOrchestrator sizes the primary leg as a percentage of capital and sizes the secondary leg via the estimated hedge ratio; order execution, commissions and borrow costs are modeled in Exchange.

##### 3. Why cointegration indicates an arbitrage opportunity
- Intuition: two non-stationary price series can have a stationary linear combination (cointegration). If the spread is stationary (finite mean and variance), large deviations tend to revert—this expected reversion is the basis of statistical arbitrage.
- Correlation vs cointegration: correlation measures contemporaneous co-movement; cointegration implies a long-run equilibrium in levels, which justifies trading the spread.
- Risks: cointegration doesn’t guarantee quick or sufficiently large reversion to cover trading costs; your implementation models commissions and borrow costs to assess feasibility.

##### 4. Justification for the Kalman Filter for dynamic hedging
- Need for dynamic hedge: the hedge ratio (beta1) can change over time due to relative volatility or structural shifts.
- Technical advantage: the Kalman Filter provides recursive, adaptive estimates of beta0 and beta1 (state is random-walk; observation is linear), updating every tick without full re-calibration. This reduces hedging error versus a fixed beta. The KF setup and filter_update logic live inside MarketNeutralStrategy.
- Practical benefits: improved dynamic coverage, fewer batch re-estimations, faster reaction to regime changes.
- Limitations: assumes linear-Gaussian dynamics; large structural breaks or non-linearities can degrade performance and require regime-change detection.

##### 5. Expected market conditions for success
- Pairs with stable economic/fundamental relationship (same sector, similar exposures).
- Sufficient liquidity to limit slippage in both legs.
- Low-to-moderate volatility: clearer mean-reversion signals; in extreme volatility, signals can be noisy.
- Controlled costs: sensitivity to commissions and borrow fees—these are modeled in Exchange and included in performance reports.
- Not recommended during structural breaks (mergers, idiosyncratic news) without detection mechanisms.

##### 6. Implementation, validation and metrics
- Pipeline & parameters: main.py handles data loading, splits (60/20/20), Exchange and Strategy initialization, and test execution.
- Market engine & costs: Exchange models execution, fees, daily borrow costs, and tracks portfolio history and executed trades.
- Robustness testing: BackTesting class supports K-Fold evaluation to analyze stability across different time folds.
- Metrics: generate_performance_report computes Sharpe, Sortino, Max Drawdown, Calmar and trade stats to quantify cost impact and performance.
- Recommendations: sensitivity analysis on rolling_window, entry/exit thresholds and capital allocation; stress-tests with higher fees/borrow costs; monitor hedge_ratio_history and spread/z-score for signal degradation.

### Pair Selection and Methodology

### Sequential Decision Analysis Framework

**Sequential Decision Analysis (SDA) Framework**
This framework models the entire trading process as a sequential loop of observing, learning, and acting.   
* Detailed Mathematical Formulation of State-Space Model:
    The system is defined by a linear state-space model, which includes a hidden state (the hedge ratio) and a measurable observation (the asset prices).
    * State Equation (Transition): $X_t = F \cdot X_{t-1} + \omega_t$ 
        * The hidden state $X_t$ is the vector of dynamic regression coefficients: $X_t = [\beta_{0,t} \text{ (intercept)}, \beta_{1,t} \text{ (slope)}]^T$.
        * The transition matrix $F$ is set to the identity matrix (np.eye(2)), modeling the state as a random walk. This assumes the coefficients at time $t$ are the same as at $t-1$, plus some process noise $\omega_t$.
    * Observation Equation (Measurement): $Y_t = H_t \cdot X_t + \epsilon_t$
        * The observation $Y_t$ is the price of Asset 1: $P_{1,t}$.
        * The observation matrix $H_t$ is dynamic and updated at each step: $H_t = [1, P_{2,t}]$, where $P_{2,t}$ is the price of Asset 2.
        * The equation expands to: $P_{1,t} = 1 \cdot \beta_{0,t} + P_{2,t} \cdot \beta_{1,t} + \epsilon_t$, where $\epsilon_t$ is the measurement noise.

* Description of Sequential Process (predict $\rightarrow$ observe $\rightarrow$ update $\rightarrow$ decide $\rightarrow$ act $\rightarrow$ learn):
This loop is executed by the StrategyOrchestrator for each timestamp in the historical data.
    * Predict: The Kalman Filter (internally within pykalman) predicts the a priori state $[\beta_0, \beta_1]$ for time $t$.
    * Observe: The StrategyOrchestrator reads the current market prices $P_1$ and $P_2$ for the current timestamp.
    * Update (Learn): The orchestrator calls strategy.get_signals(). This method provides the new observation $P_1$ and observation matrix $[1, P_2]$ to the kf_hedge.filter_update function. The filter calculates the a posteriori (updated) state, effectively "learning" the new, most likely hedge ratio.
    * Decide: get_signals() uses the updated state to compute the current spread and its Z-score. It compares this Z-score against entry_threshold and exit_threshold to generate a signal: (1: Open Long), (0: Open Short), (-2: Close Position).
    * Act: The StrategyOrchestrator receives the signal. If it's an entry signal (1 or 0) and the position is neutral, it calls _open_position. If it's an exit signal (-2) and a position is open, it calls _close_position. These methods execute trades via exchange.execute_trade.
* Q and R Matrix Selection with Justification:
    * R (Observation Covariance): observation_covariance = 1.0. This represents the variance of the measurement noise $\epsilon_t$. A value of 1.0 is a standard baseline, implying moderate uncertainty in the observed price relationship ($P_1$ vs. $P_2$) that isn't captured by the hedge ratio.
    * Q (Transition Covariance): transition_covariance = np.eye(2) * 1e-5. This represents the variance of the process noise $\omega_t$. This critically small value is a key design choice. It signifies a strong belief that the hedge ratio coefficients $[\beta_0, \beta_1]$ are highly stable and evolve very slowly. This makes the filter smooth and resistant to short-term market noise.

### Kalman Filter Implementation

**Kalman Filter Implementation**
 
This section details the practical setup and operation of the filter.
* Initialization Procedures:The filter is initialized with a "prior belief" about the state:
    * initial_state_mean = np.array([0.0, 1.0]): The filter's initial belief is that the intercept is 0 and the slope is 1. This is a neutral and logical starting point for a pair relationship.
    
    * initial_state_covariance = np.eye(2) * 1.0: This initializes the filter with moderate uncertainty about its initial belief, allowing it to converge quickly to the true parameters as it processes new data.
* Parameter Estimation Methodology:The state parameters ($\beta_0, \beta_1$) are estimated recursively and online. Unlike a static OLS regression, the filter updates its parameter estimates at every single timestamp. This is achieved via the kf_hedge.filter_update method, which optimally blends the previous state prediction with the new observation to produce the most probable current state.
* Reestimation Schedule and Validation Approach:
    * Reestimation Schedule: The reestimation is continuous. The hedge ratio is re-calculated at every time step (e.g., daily) as new price data becomes available.
    * Validation Approach: The overall strategy's performance is validated using the BackTesting class. This class implements K-Fold cross-validation. Crucially, it sets shuffle=False to preserve the temporal order of the data, which is essential for time-series analysis. This allows for performance assessment across multiple distinct "folds" or time periods.

### Trading Strategy Logic

**Trading Strategy Logic**

This section describes how the filter's outputs are translated into discrete trading actions.
* Z-score Definition (using Kalman Filter Spread):Note: While VECM is a valid approach, this implementation uses a spread dynamically derived from the Kalman Filter's state.
    * Spread Calculation: At each step $t$, the spread is calculated using the updated coefficients from the filter:$\text{Spread}_t = P_{1,t} - (\beta_{0,t} + \beta_{1,t} \cdot P_{2,t})$.
    * Z-score Calculation: This spread value is appended to a history (self.spread_history). The Z-score is then calculated by standardizing the current spread relative to its own rolling history (defined by rolling_window):$Z_t = \frac{\text{Spread}_t - \text{rolling\_mean}(\text{Spreads})}{\text{rolling\_std}(\text{Spreads})}$
* Optimal Entry and Exit Z-score Policy Found:The trading policy is governed by symmetric thresholds:
    * Entry (Open Position):
        * If z_score < -self.entry_threshold (e.g., -2.0): Generate Signal 1 (Buy Spread: Long P1, Short P2).
        * If z_score > self.entry_threshold (e.g., +2.0): Generate Signal 0 (Sell Spread: Short P1, Long P2).
    * Exit (Close Position):
        * If abs(z_score) < self.exit_threshold (e.g., 0.5): Generate Signal -2 (Reversion to mean), triggering a call to _close_position.
    * Cost Treatment: Commissions and Borrow Rates:The Exchange class realistically models transaction frictions.
        * Commissions: A fee_rate (e.g., 0.125%) is applied to the total USD value of every transaction leg (both opening and closing trades). This cost is immediately deducted from the cash balance.
        
        * Borrow Rates: At every time step, the update_exchange_status method calculates the total USD value of all short positions. A borrow_rate_daily (e.g., 0.25%) is applied to this value, and the resulting borrow_cost is deducted from the cash balance.

### Results and Performance Analysis

### Conclusions